# 04 — Analyse Accessibilité des gares

Notebook pour explorer `accessibilite-en-gare.csv` et produire :
- filtrage par niveau d'accessibilité
- stats par département
- top gares/communes accessibles
- widgets interactifs


In [2]:

import os, warnings
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets, HBox, VBox
warnings.filterwarnings('ignore')

# chemins par défaut (modifiables)
default_paths = [
    os.path.join("data","raw","accessibilite-en-gare.csv"),
    os.path.join("..","data","raw","accessibilite-en-gare.csv"),
    r"C:/Users/Victor/DataScience/Projet-Data-science-Investissement-immobilier/accessibilite-en-gare.csv"
]

csv_path = None
for p in default_paths:
    if os.path.exists(p):
        csv_path = p; break

print('Chemin utilisé pour accessibilite:', csv_path or "Aucun fichier trouvé automatiquement — modifiez csv_path si besoin")


ModuleNotFoundError: No module named 'ipywidgets'

## Chargement et typage

In [ ]:

if csv_path is None:
    raise FileNotFoundError("Aucun fichier accessibilite-en-gare.csv trouvé. Déposez le fichier dans data/raw/")

df_raw = pd.read_csv(csv_path, sep=';', engine='python', dtype=str)
print("Taille initiale:", df_raw.shape)
display(df_raw.head(3))
print("Colonnes:", list(df_raw.columns))


In [ ]:

# conversion accessibility_level_id
col_acc = None
for c in df_raw.columns:
    if 'accessibility_level' in c.lower() or 'accessibilit' in c.lower():
        col_acc = c; break
if col_acc is None:
    raise ValueError("Impossible de trouver la colonne d'accessibilité dans le fichier.")

df_raw[col_acc] = pd.to_numeric(df_raw[col_acc], errors='coerce')
print('Colonne accessibilité détectée :', col_acc)
df = df_raw.copy()


## Filtrage — niveau minimal d'accessibilité

In [ ]:

# distribution
print("Distribution niveaux accessibilité:")
display(df[col_acc].value_counts().sort_index())

# Par défaut, on garde niveaux >=3
df_filtered = df[df[col_acc] >= 3].copy()
print("Gares avec niveau >=3 :", len(df_filtered))


## Visualisations (Matplotlib)
- Histogramme des niveaux
- Top communes par nombre de gares accessibles


In [ ]:

# histogramme des niveaux (toutes gares)
plt.figure()
plt.hist(df[col_acc].dropna().astype(int), bins=range(int(df[col_acc].min()), int(df[col_acc].max())+2))
plt.title('Histogramme des niveaux d accessibilité')
plt.xlabel('Niveau accessibilité'); plt.ylabel('Nombre de gares')
plt.show()


In [ ]:

# Top communes avec le plus de gares accessibles (colonne commune possible)
comm_col = None
for c in ['stop_name','commune','nom_commune','town','locality']:
    if c in df.columns:
        comm_col = c; break
if comm_col is None:
    # fallback sur stop_point_id ou autre
    comm_col = 'stop_point_id'

top_comm = df_filtered.groupby(comm_col).size().reset_index(name='count').sort_values('count', ascending=False).head(20)
display(top_comm)
plt.figure()
plt.bar(top_comm[comm_col].astype(str), top_comm['count'])
plt.title('Top gares / commune (niveau >=3)')
plt.xlabel('Commune/Gare'); plt.ylabel('Nombre de gares')
plt.xticks(rotation=45, ha='right')
plt.show()


## Widgets interactifs
- Slider niveau minimal (1–5)
- Filtre texte sur nom de gare / commune
- Export CSV des gares filtrées


In [ ]:

level_slider = widgets.IntSlider(value=3, min=int(df[col_acc].min()), max=int(df[col_acc].max()), step=1, description='Niveau >=')
text_filter = widgets.Text(value='', description='Filtre nom:')
topn = widgets.IntSlider(value=20, min=5, max=200, step=5, description='Top N:')

def explore(level_min, filt_text, topn_val):
    d = df.copy()
    d = d[d[col_acc] >= level_min]
    if filt_text.strip():
        mask = d.apply(lambda row: filt_text.lower() in ' '.join([str(row.get(c,'')) for c in d.columns if isinstance(row.get(c,''), str)]).lower(), axis=1)
        d = d[mask]
    # agrégation par commune/gare
    grp = d.groupby(comm_col).size().reset_index(name='count').sort_values('count', ascending=False)
    display(grp.head(topn_val))
    print(f"Total gares correspondant: {len(d)}")
    return d, grp

out = widgets.interactive_output(explore, {'level_min': level_slider, 'filt_text': text_filter, 'topn_val': topn})
display(HBox([level_slider, text_filter, topn]))
display(out)


## Export des gares filtrées

In [ ]:

def export_gare(df_out, fname='gares_accessibles_filtered.csv'):
    outp = os.path.join('data','clean', fname)
    os.makedirs(os.path.dirname(outp), exist_ok=True)
    df_out.to_csv(outp, index=False, encoding='utf-8-sig')
    print('Exporté →', outp)

# Exemple:
# d, grp = explore(3, '', 20)
# export_gare(d)
print('Cellule prête pour export.')
